In [1]:
import requests
from pyscicat.client import ScicatClient
from pyscicat.model import RawDataset, Ownable
from datetime import datetime


In [2]:
BASE_URL="http://localhost:3000/api/v3/"
SCICAT_USER="ingestor"
SCICAT_PASSWORD="aman"

Adding a dataset to Scicat with the `ingestor` role. The dataset belongs to the `ingestor` and the `ingestor` group is an `admin` group. 

We test the following methods with both `pyscicat` and the `requests` package:
* login on the datasets endpoint
* post on the datasets endpoint
* get on the datasets endpoint

In [3]:
# Check that you can login to the Scicat API with requests 
r = requests.request("post", BASE_URL + 'auth/login', json=dict(username='ingestor', password='aman'))
token = r.json()['id']
print(r.status_code)

# Check you can make a connection with the Scicat Client
sci_clie = ScicatClient(base_url=BASE_URL,
                        token=None,
                        username=SCICAT_USER,
                        password=SCICAT_PASSWORD)


201


In [4]:
# Create Dataset with the admin owner group and as the ingestor role
ownable = Ownable(ownerGroup="ingestor", accessGroups=[])
dataset_a = RawDataset(
    datasetName="a new guide book",
    path="/foo/bar",
    size=42,
    packedSize=0,
    owner=SCICAT_USER,
    contactEmail="slartibartfast@magrathea.org",
    creationLocation="Magrathea",
    creationTime=datetime.isoformat(datetime.now()),
    instrumentId="earth",
    proposalId="deepthought",
    dataFormat="planet",
    principalInvestigator="A. Mouse",
    sourceFolder="/foo/bar",
    scientificMetadata={"type": "string", "value": {"a": "field"}},
    sampleId="gargleblaster",
    type="raw",
    ownerEmail="scicatingestor@your.site",
    sourceFolderHost="s3.heartofgold.org",
    endTime=datetime.isoformat(datetime.now()),
    techniques=[],
    numberOfFiles=0,
    numberOfFilesArchived=0,
    **ownable.dict()
)

dataset_b = RawDataset(
    datasetName="a BIGGER guide book",
    path="/foo/bar",
    size=42,
    packedSize=0,
    owner=SCICAT_USER,
    contactEmail="slartibartfast@magrathea.org",
    creationLocation="Magrathea",
    creationTime=datetime.isoformat(datetime.now()),
    instrumentId="earth",
    proposalId="deepthought",
    dataFormat="planet",
    principalInvestigator="A. Mouse",
    sourceFolder="/foo/bar",
    scientificMetadata={"type": "string", "value": {"a": "field"}},
    sampleId="gargleblaster",
    type="raw",
    ownerEmail="scicatingestor@your.site",
    sourceFolderHost="s3.heartofgold.org",
    endTime=datetime.isoformat(datetime.now()),
    techniques=[],
    numberOfFiles=0,
    numberOfFilesArchived=0,
    **ownable.dict()
)




In [5]:
# Can you upload a dataset with requests as an ingestor role ?
r = requests.request('post',BASE_URL + 'Datasets', json=dataset_a.dict(exclude_none=True),
                          headers={'Authorization': f'Bearer {token}'})

print(r.text)

{"createdBy":"ingestor","updatedBy":"ingestor","ownerGroup":"ingestor","accessGroups":[],"owner":"ingestor","ownerEmail":"scicatingestor@your.site","contactEmail":"slartibartfast@magrathea.org","sourceFolder":"/foo/bar","sourceFolderHost":"s3.heartofgold.org","size":42,"packedSize":0,"numberOfFiles":0,"numberOfFilesArchived":0,"creationTime":"2023-04-04T12:45:33.910Z","type":"raw","keywords":[],"datasetName":"a new guide book","isPublished":false,"datasetlifecycle":{"archivable":true,"retrievable":false,"publishable":false,"dateOfDiskPurging":"2023-04-04T11:56:40.368Z","publishedOn":"2023-04-04T11:56:40.368Z","isOnCentralDisk":true,"archiveStatusMessage":"datasetCreated","retrieveStatusMessage":"","retrieveIntegrityCheck":false,"_id":"642c1bf03fc7fbae00ff5c84","archiveRetentionTime":"2023-04-02T00:00:00.000Z","dateOfPublishing":"2023-04-02T00:00:00.000Z"},"techniques":[],"sharedWith":[],"scientificMetadata":{"type":"string","value":{"a":"field"}},"principalInvestigator":"A. Mouse","end

In [6]:
# Can you upload a dataset with pyscict as an ingestor role ?
sci_clie.upload_new_dataset(dataset_b)

'PID.SAMPLE.PREFIX/6493cd66-1859-46bd-8163-de838ea944cf'

In [7]:
# Can you get your dataset back as an admin role?
r = requests.request('get',BASE_URL + 'Datasets', json= {"where": {"owner": "ingestor"}},
                          headers={'Authorization': f'Bearer {token}'})

print(r.text)

[{"_id":"PID.SAMPLE.PREFIX/d3695509-415d-4cc1-82e3-ca68d6442392","createdBy":"ingestor","updatedBy":"ingestor","ownerGroup":"ingestor","accessGroups":[],"owner":"ingestor","ownerEmail":"scicatingestor@your.site","contactEmail":"slartibartfast@magrathea.org","sourceFolder":"/foo/bar","sourceFolderHost":"s3.heartofgold.org","size":42,"packedSize":0,"numberOfFiles":0,"numberOfFilesArchived":0,"creationTime":"2023-04-04T12:08:08.733Z","type":"raw","keywords":[],"datasetName":"a new guide book","isPublished":false,"datasetlifecycle":{"archivable":true,"retrievable":false,"publishable":false,"dateOfDiskPurging":"2023-04-04T11:56:40.368Z","publishedOn":"2023-04-04T11:56:40.368Z","isOnCentralDisk":true,"archiveStatusMessage":"datasetCreated","retrieveStatusMessage":"","retrieveIntegrityCheck":false,"_id":"642c132a3fc7fbae00ff5c29","archiveRetentionTime":"2023-04-02T00:00:00.000Z","dateOfPublishing":"2023-04-02T00:00:00.000Z"},"techniques":[],"sharedWith":[],"scientificMetadata":{"type":"string

In [8]:
# Can you get your dataset back with an admin role from pyscicat?
sci_clie.get_datasets({"owner":"ingestor"})

AttributeError: 'str' object has no attribute 'get'

We will now add data from a user `user1` who is part of `group1` . The `user1` does not have an ingestor role and the `group1` is not part of `admin`.

We will try to upload the dataset with `pyscicat` and `requests` packages as the `ingestor` adn `user1` and GET the datasets as `user1` and the `ingestor`

Expected behaviour:
* write and read as `ingestor`
* read only as `user1`

In [9]:
#ingest a data set from a user

ownable = Ownable(ownerGroup="group1", accessGroups=[])
dataset_user = RawDataset(
    datasetName="Intergalatic Highway",
    path="/foo/bar",
    size=42,
    packedSize=0,
    owner="user1",
    contactEmail="jeltz@vogons.org",
    creationLocation="Vogsphere",
    creationTime=datetime.isoformat(datetime.now()),
    instrumentId="bulldozer",
    proposalId="Demolition of Earth",
    dataFormat="highway",
    principalInvestigator="A. Vogon",
    sourceFolder="/foo/bar",
    scientificMetadata={"type": "string", "value": {"a": "field"}},
    sampleId="",
    ownerEmail="jeltz@vogons.org",
    sourceFolderHost="s3.vogscloud.org",
    endTime=datetime.isoformat(datetime.now()),
    techniques=[],
    numberOfFiles=0,
    numberOfFilesArchived=0,
    **ownable.dict()
)




In [11]:
# Can you upload as ingestor?
r = requests.request('post',BASE_URL + 'Datasets', json=dataset_user.dict(exclude_none=True),
                          headers={'Authorization': f'Bearer {token}'})
print(r.text)

# Upload with pyscicat
sci_clie.upload_new_dataset(dataset_user)

{"statusCode":403,"message":"Unauthorized to create this dataset","error":"Forbidden"}


ScicatCommError: Error in operation datasets_create: Forbidden

In [10]:
# Can you upload as a user?
r = requests.request("post", BASE_URL + 'auth/login', json=dict(username='user1',
                                                                password='a609316768619f154ef58db4d847b75e'))
user_token = r.json()['id']
print(r.status_code)
r = requests.request('post',BASE_URL + 'Datasets', json=dataset_user.dict(exclude_none=True),
                          headers={'Authorization': f'Bearer {user_token}'})
print(r.text)

# Upload with Pyscicat


sci_user = ScicatClient(base_url=BASE_URL,
                        token=None,
                        username='user1',
                        password='a609316768619f154ef58db4d847b75e')
sci_user.upload_new_dataset(dataset_user)

201
{"createdBy":"user1","updatedBy":"user1","ownerGroup":"group1","accessGroups":[],"owner":"user1","ownerEmail":"jeltz@vogons.org","contactEmail":"jeltz@vogons.org","sourceFolder":"/foo/bar","sourceFolderHost":"s3.vogscloud.org","size":42,"packedSize":0,"numberOfFiles":0,"numberOfFilesArchived":0,"creationTime":"2023-04-04T12:45:48.754Z","type":"raw","keywords":[],"datasetName":"Intergalatic Highway","isPublished":false,"datasetlifecycle":{"archivable":true,"retrievable":false,"publishable":false,"dateOfDiskPurging":"2023-04-04T11:56:40.368Z","publishedOn":"2023-04-04T11:56:40.368Z","isOnCentralDisk":true,"archiveStatusMessage":"datasetCreated","retrieveStatusMessage":"","retrieveIntegrityCheck":false,"_id":"642c1bfe3fc7fbae00ff5ca1","archiveRetentionTime":"2023-04-02T00:00:00.000Z","dateOfPublishing":"2023-04-02T00:00:00.000Z"},"techniques":[],"sharedWith":[],"scientificMetadata":{"type":"string","value":{"a":"field"}},"principalInvestigator":"A. Vogon","endTime":"2023-04-04T12:45:4

'PID.SAMPLE.PREFIX/e212c212-aa8a-4c86-aefa-d52ef7df16ef'

In [13]:
# Can you read as ingestor?
r = requests.request('get',BASE_URL + 'Datasets', json= {"where": {"owner": "user1"}},
                          headers={'Authorization': f'Bearer {token}'})

print(r.text)

[{"_id":"PID.SAMPLE.PREFIX/d3695509-415d-4cc1-82e3-ca68d6442392","createdBy":"ingestor","updatedBy":"ingestor","ownerGroup":"ingestor","accessGroups":[],"owner":"ingestor","ownerEmail":"scicatingestor@your.site","contactEmail":"slartibartfast@magrathea.org","sourceFolder":"/foo/bar","sourceFolderHost":"s3.heartofgold.org","size":42,"packedSize":0,"numberOfFiles":0,"numberOfFilesArchived":0,"creationTime":"2023-04-04T12:08:08.733Z","type":"raw","keywords":[],"datasetName":"a new guide book","isPublished":false,"datasetlifecycle":{"archivable":true,"retrievable":false,"publishable":false,"dateOfDiskPurging":"2023-04-04T11:56:40.368Z","publishedOn":"2023-04-04T11:56:40.368Z","isOnCentralDisk":true,"archiveStatusMessage":"datasetCreated","retrieveStatusMessage":"","retrieveIntegrityCheck":false,"_id":"642c132a3fc7fbae00ff5c29","archiveRetentionTime":"2023-04-02T00:00:00.000Z","dateOfPublishing":"2023-04-02T00:00:00.000Z"},"techniques":[],"sharedWith":[],"scientificMetadata":{"type":"string

In [15]:
# Can you read as user?
r = requests.request('get',BASE_URL + 'Datasets', json= {"where": {"owner": "user1"}},
                          headers={'Authorization': f'Bearer {user_token}'})

print(r.text)


[{"_id":"PID.SAMPLE.PREFIX/c66c8f30-29a7-4a94-9ee9-a3661fb67908","createdBy":"user1","updatedBy":"user1","ownerGroup":"group1","accessGroups":[],"owner":"user1","ownerEmail":"jeltz@vogons.org","contactEmail":"jeltz@vogons.org","sourceFolder":"/foo/bar","sourceFolderHost":"s3.vogscloud.org","size":42,"packedSize":0,"numberOfFiles":0,"numberOfFilesArchived":0,"creationTime":"2023-04-04T12:29:03.957Z","type":"raw","keywords":[],"datasetName":"Intergalatic Highway","isPublished":false,"datasetlifecycle":{"archivable":true,"retrievable":false,"publishable":false,"dateOfDiskPurging":"2023-04-04T11:56:40.368Z","publishedOn":"2023-04-04T11:56:40.368Z","isOnCentralDisk":true,"archiveStatusMessage":"datasetCreated","retrieveStatusMessage":"","retrieveIntegrityCheck":false,"_id":"642c18ec3fc7fbae00ff5c70","archiveRetentionTime":"2023-04-02T00:00:00.000Z","dateOfPublishing":"2023-04-02T00:00:00.000Z"},"techniques":[],"sharedWith":[],"scientificMetadata":{"type":"string","value":{"a":"field"}},"pri